In [ ]:
import numpy as np
import pandas as pd
import json
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
import tensorflow as tf
from tensorflow.keras.layers import Input, Embedding, Dense, Concatenate, Flatten, GlobalAveragePooling1D
from tensorflow.keras.models import Model

### Data Loading

In [ ]:
order_data = pd.read_csv("order_data.csv")
customer_data = pd.read_csv("customer_data.csv")
test_data = pd.read_csv('test_data_question.csv')
# store_data = pd.read_csv("store_data.csv") # Not directly used in the current preprocessing

In [ ]:
order_data.info()

In [ ]:
customer_data.info()

### EDA

1. Initial Data Cleaning and Feature Engineering

In [ ]:
# Handle null values for CUSTOMER_TYPE
customer_data['CUSTOMER_TYPE'] = customer_data['CUSTOMER_TYPE'].fillna('Registered')

# Convert ORDER_CREATED_DATE to datetime and extract features
order_data['ORDER_CREATED_DATE'] = pd.to_datetime(order_data['ORDER_CREATED_DATE'])
order_data['YEAR'] = order_data['ORDER_CREATED_DATE'].dt.year
order_data['MONTH'] = order_data['ORDER_CREATED_DATE'].dt.month
order_data['DAY'] = order_data['ORDER_CREATED_DATE'].dt.day
order_data.drop('ORDER_CREATED_DATE', axis=1, inplace=True)

2. Efficient JSON Parsing and Flattening

In [ ]:
# Define pattern for items to exclude
pattern_to_exclude = r"^(Order|Delivery Fee|Sub Box|Unavailable Item|Plastic|Extra Sauce|Ketchup Pack|Seasoning Pack)"

# Parse 'ORDERS' JSON directly into a new column
order_data['PARSED_ORDERS'] = order_data['ORDERS'].apply(json.loads)

# Extract 'item_details' list
order_data['ITEM_DETAILS_LIST'] = order_data['PARSED_ORDERS'].apply(lambda x: x['orders'][0]['item_details'])

# Explode the DataFrame to have one row per item within an order
# Select only necessary columns before exploding to minimize memory footprint
# and later merge with customer data.
processed_items = order_data[[
    'CUSTOMER_ID', 'STORE_NUMBER', 'ORDER_ID',
    'ORDER_CHANNEL_NAME', 'ORDER_SUBCHANNEL_NAME', 'ORDER_OCCASION_NAME',
    'YEAR', 'MONTH', 'DAY', 'ITEM_DETAILS_LIST'
]].explode('ITEM_DETAILS_LIST')

# Extract item details into separate columns using .apply(get) - efficient
processed_items['ITEM_NAME'] = processed_items['ITEM_DETAILS_LIST'].apply(lambda x: x.get('item_name'))
processed_items['ITEM_PRICE'] = processed_items['ITEM_DETAILS_LIST'].apply(lambda x: x.get('item_price')).astype(float)
processed_items['ITEM_QUANTITY'] = processed_items['ITEM_DETAILS_LIST'].apply(lambda x: x.get('item_quantity')).astype(int)

# Filter out excluded items (using the regex pattern)
processed_items = processed_items[~processed_items['ITEM_NAME'].str.contains(pattern_to_exclude, na=False, regex=True)]

# Drop the intermediate list column
processed_items.drop(columns=['ITEM_DETAILS_LIST'], inplace=True)

train_data_dl = pd.merge(
    processed_items,
    customer_data[['CUSTOMER_ID', 'CUSTOMER_TYPE']],
    on='CUSTOMER_ID',
    how='left'
)

3. Calculate Cart-Level Numerical Features

In [ ]:
train_data_dl['CART_AVG_ITEM_PRICE'] = train_data_dl.groupby('ORDER_ID')['ITEM_PRICE'].transform('mean')
train_data_dl['CART_TOTAL_QUANTITY'] = train_data_dl.groupby('ORDER_ID')['ITEM_QUANTITY'].transform('sum')
train_data_dl['CART_NUM_UNIQUE_ITEMS'] = train_data_dl.groupby('ORDER_ID')['ITEM_NAME'].transform('nunique')

4. Encoding Categorical Features

In [ ]:
# Initialize all encoders
item_name_encoder = LabelEncoder()
order_id_encoder = LabelEncoder()
customer_id_encoder = LabelEncoder()
customer_type_encoder = LabelEncoder()
order_channel_name_encoder = LabelEncoder()
order_subchannel_name_encoder = LabelEncoder()
order_occasion_name_encoder = LabelEncoder()
scaler = MinMaxScaler()

# Fit ALL LabelEncoders on the entire relevant columns from train_data_dl
train_data_dl['ITEM_ID'] = item_name_encoder.fit_transform(train_data_dl['ITEM_NAME'])
train_data_dl['ORDER_ID_ENCODED'] = order_id_encoder.fit_transform(train_data_dl['ORDER_ID'])
train_data_dl['CUSTOMER_ID_ENCODED'] = customer_id_encoder.fit_transform(train_data_dl['CUSTOMER_ID'])
train_data_dl['ORDER_CHANNEL_NAME_ENCODED'] = order_channel_name_encoder.fit_transform(train_data_dl['ORDER_CHANNEL_NAME'])
train_data_dl['ORDER_SUBCHANNEL_NAME_ENCODED'] = order_subchannel_name_encoder.fit_transform(train_data_dl['ORDER_SUBCHANNEL_NAME'])
train_data_dl['ORDER_OCCASION_NAME_ENCODED'] = order_occasion_name_encoder.fit_transform(train_data_dl['ORDER_OCCASION_NAME'])

# For CUSTOMER_TYPE, fit on unique values from the original customer_data to maintain consistency and then transform.
customer_type_encoder.fit(customer_data['CUSTOMER_TYPE'].unique())
train_data_dl['CUSTOMER_TYPE_ENCODED'] = customer_type_encoder.transform(train_data_dl['CUSTOMER_TYPE'])

# Create mappings for decoding later
customer_id_map_dl = {idx: original_id for idx, original_id in enumerate(customer_id_encoder.classes_)}
item_name_map_dl = {idx: original_name for idx, original_name in enumerate(item_name_encoder.classes_)}
customer_type_map = {idx: original_type for idx, original_type in enumerate(customer_type_encoder.classes_)}
order_channel_map = {idx: name for idx, name in enumerate(order_channel_name_encoder.classes_)}
order_subchannel_map = {idx: name for idx, name in enumerate(order_subchannel_name_encoder.classes_)}
order_occasion_map = {idx: name for idx, name in enumerate(order_occasion_name_encoder.classes_)}



5. Calculate MAX_CART_SIZE and CART_ITEM_IDS_ENCODED_PADDED

In [ ]:
# Determine MAX_CART_SIZE from the processed data
MAX_CART_SIZE = train_data_dl.groupby('ORDER_ID')['ITEM_NAME'].nunique().max()
print(f"Determined MAX_CART_SIZE: {MAX_CART_SIZE}")

# Helper function for padding (moved outside to be simpler)
def pad_encoded_item_ids(item_ids, max_size, padding_value=0):
    padded_ids = np.full(max_size, padding_value, dtype=int)
    actual_len = len(item_ids)
    padded_ids[:min(actual_len, max_size)] = item_ids[:min(actual_len, max_size)]
    return padded_ids

# Pre-calculate padded item IDs for each unique ORDER_ID
# We'll use the item_name_encoder directly here.
order_item_encoded_ids = train_data_dl.groupby('ORDER_ID')['ITEM_ID'].apply(list)

# Apply padding to each list of encoded item IDs
padded_item_ids_per_order_series = order_item_encoded_ids.apply(
    lambda x: pad_encoded_item_ids(np.array(x), MAX_CART_SIZE)
)

# Map these pre-calculated padded IDs back to the original train_data_dl DataFrame
train_data_dl['CART_ITEM_IDS_ENCODED_PADDED'] = train_data_dl['ORDER_ID'].map(padded_item_ids_per_order_series)


6. Scale Numerical Features

In [ ]:
numerical_features = ['ITEM_PRICE', 'CART_AVG_ITEM_PRICE', 'CART_TOTAL_QUANTITY', 'CART_NUM_UNIQUE_ITEMS']
train_data_dl[numerical_features] = scaler.fit_transform(train_data_dl[numerical_features])

7. Define Input Features and Target, then Split

In [ ]:
categorical_cols = [
    'CUSTOMER_ID_ENCODED', 'ITEM_ID', 'ORDER_CHANNEL_NAME_ENCODED',
    'ORDER_SUBCHANNEL_NAME_ENCODED', 'ORDER_OCCASION_NAME_ENCODED', 'CUSTOMER_TYPE_ENCODED'
]

numerical_cols = ['ITEM_PRICE', 'CART_AVG_ITEM_PRICE', 'CART_TOTAL_QUANTITY', 'CART_NUM_UNIQUE_ITEMS']

cart_item_embedding_col = ['CART_ITEM_IDS_ENCODED_PADDED']

target_col = 'ITEM_QUANTITY'

X = train_data_dl[categorical_cols + numerical_cols + cart_item_embedding_col]
Y = train_data_dl[target_col]

# Split into training and validation sets
X_train, X_val, Y_train, Y_val = train_test_split(X, Y, test_size=0.2, random_state=42)

print(f"\nTraining data shape with all features: {X_train.shape}, Target shape: {Y_train.shape}")

#### Hybrid Deep Learning Model

1. Define Vocabulary Sizes & Embedding Dimensions

In [ ]:
# --- Define Vocabulary Sizes (Cardinalities) ---
num_customers = len(customer_id_encoder.classes_)
num_items = len(item_name_encoder.classes_)
num_order_channels = len(order_channel_name_encoder.classes_)
num_order_subchannels = len(order_subchannel_name_encoder.classes_)
num_order_occasions = len(order_occasion_name_encoder.classes_)
num_customer_types = len(customer_type_encoder.classes_)

# --- Define Embedding Dimensions (Hyperparameters) ---
customer_embedding_dim = min(50, num_customers // 2) if num_customers > 1 else 1
item_embedding_dim = min(50, num_items // 2) if num_items > 1 else 1
channel_embedding_dim = min(10, num_order_channels // 2) if num_order_channels > 1 else 1
subchannel_embedding_dim = min(10, num_order_subchannels // 2) if num_order_subchannels > 1 else 1
occasion_embedding_dim = min(10, num_order_occasions // 2) if num_order_occasions > 1 else 1
customer_type_embedding_dim = min(10, num_customer_types // 2) if num_customer_types > 1 else 1

2. Model Architecture

In [ ]:
# 1. Input Layers for each feature
customer_id_input = Input(shape=(1,), name='customer_id_input')
item_id_input = Input(shape=(1,), name='item_id_input')
order_channel_input = Input(shape=(1,), name='order_channel_input')
order_subchannel_input = Input(shape=(1,), name='order_subchannel_input')
order_occasion_input = Input(shape=(1,), name='order_occasion_input')
customer_type_input = Input(shape=(1,), name='customer_type_input')
item_price_input = Input(shape=(1,), name='item_price_input')
cart_items_input = Input(shape=(MAX_CART_SIZE,), name='cart_items_input')

# 2. Embedding Layers for categorical features
customer_embedding = Embedding(input_dim=num_customers, output_dim=customer_embedding_dim, name='customer_embedding')(customer_id_input)
item_embedding = Embedding(input_dim=num_items, output_dim=item_embedding_dim, name='item_embedding')(item_id_input)
order_channel_embedding = Embedding(input_dim=num_order_channels, output_dim=channel_embedding_dim, name='order_channel_embedding')(order_channel_input)
order_subchannel_embedding = Embedding(input_dim=num_order_subchannels, output_dim=subchannel_embedding_dim, name='order_subchannel_embedding')(order_subchannel_input)
order_occasion_embedding = Embedding(input_dim=num_order_occasions, output_dim=occasion_embedding_dim, name='order_occasion_embedding')(order_occasion_input)
customer_type_embedding = Embedding(input_dim=num_customer_types, output_dim=customer_type_embedding_dim, name='customer_type_embedding')(customer_type_input)
cart_items_embedded = Embedding(input_dim=num_items, output_dim=item_embedding_dim, name='cart_items_embedding')(cart_items_input)

# Average pooling for cart items
cart_items_pooled = GlobalAveragePooling1D(name='cart_items_pooled_embedding')(cart_items_embedded) 

# Flatten other embedding outputs
customer_vec = Flatten()(customer_embedding)
item_vec = Flatten()(item_embedding)
order_channel_vec = Flatten()(order_channel_embedding)
order_subchannel_vec = Flatten()(order_subchannel_embedding)
order_occasion_vec = Flatten()(order_occasion_embedding)
customer_type_vec = Flatten()(customer_type_embedding)

# 3. Concatenate all features (flattened embeddings + numerical inputs + pooled cart items)
concatenated_features = Concatenate()(
    [customer_vec, item_vec, order_channel_vec, order_subchannel_vec,
     order_occasion_vec, customer_type_vec,
     item_price_input, cart_items_pooled] # Added cart_items_pooled here
)

# 4. Deep Neural Network (DNN) layers
dense_1 = Dense(128, activation='relu')(concatenated_features)
dense_2 = Dense(64, activation='relu')(dense_1)
dense_3 = Dense(32, activation='relu')(dense_2)

# 5. Output Layer: Predict ITEM_QUANTITY (regression task)
output_layer = Dense(1, activation='relu', name='item_quantity_prediction')(dense_3)

3. Initialization and Compilation

In [ ]:
# Create the model
model = Model(
    inputs=[customer_id_input, item_id_input, order_channel_input, order_subchannel_input,
            order_occasion_input, customer_type_input,
            item_price_input, cart_items_input], # Updated inputs
    outputs=output_layer
)

# Compile the model
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3), 
    loss=tf.keras.losses.Huber(), 
    metrics=[
            tf.keras.metrics.MeanAbsoluteError(name="mae"),
            tf.keras.metrics.RootMeanSquaredError(name="rmse"),
        ])

print("Deep Learning Model Summary (with Padded Cart Items):")
model.summary()

4. Training

In [ ]:
# Map X_train, X_val to the dictionary format expected by Keras
def to_input_dict(df):
    return {
        'customer_id_input': df['CUSTOMER_ID_ENCODED'],
        'item_id_input': df['ITEM_ID'],
        'order_channel_input': df['ORDER_CHANNEL_NAME_ENCODED'],
        'order_subchannel_input': df['ORDER_SUBCHANNEL_NAME_ENCODED'],
        'order_occasion_input': df['ORDER_OCCASION_NAME_ENCODED'],
        'customer_type_input': df['CUSTOMER_TYPE_ENCODED'],
        'item_price_input': df['ITEM_PRICE'],
        'cart_items_input': np.array(df['CART_ITEM_IDS_ENCODED_PADDED'].tolist()) # Convert list of lists to numpy array
    }

train_inputs = to_input_dict(X_train)
val_inputs = to_input_dict(X_val)

train_inputs = to_input_dict(X_train)
val_inputs = to_input_dict(X_val)

training = False
if training:
    print("\nTraining Deep Learning Model...")
    history = model.fit(
        train_inputs,
        Y_train,
        epochs=5, # Reduced epochs for quick demonstration
        batch_size=64,
        validation_data=(val_inputs, Y_val),
        verbose=1
    )
    print("Model Training Complete.")
    model.save_weights("model6_v1.weights.h5")
else:
    print("\nLoading Deep Learning Model weights...")
    model.load_weights("model6_v1.weights.h5")

5. Loss & Metrics visualization

In [ ]:
def plot_training_history(history):
    """
    Plots training and validation metrics from a Keras History object.
    """
    hist = history.history
    metrics = [m for m in hist.keys() if not m.startswith('val_')]

    for metric in metrics:
        plt.figure(figsize=(8, 5))
        plt.plot(hist[metric], label=f'Training {metric}')
        if f'val_{metric}' in hist:
            plt.plot(hist[f'val_{metric}'], label=f'Validation {metric}')
        plt.xlabel('Epoch')
        plt.ylabel(metric.capitalize())
        plt.title(f'Training and Validation {metric.capitalize()}')
        plt.legend()
        plt.grid(True)
        plt.show()

# Example usage:
plot_training_history(history)

In [ ]:
def pad_cart_items(encoded_item_ids, max_size, padding_value=0):
    """Pads a list of encoded item IDs to a fixed size."""
    if len(encoded_item_ids) >= max_size:
        return encoded_item_ids[:max_size]
    else:
        return encoded_item_ids + [padding_value] * (max_size - len(encoded_item_ids))

In [ ]:
item_features = (
    train_data_dl
    .groupby('ITEM_NAME', as_index=True)
    .agg({'ITEM_PRICE': 'first', 'ITEM_QUANTITY': 'first'})
    .to_dict(orient='index')
)

In [ ]:
# --- Helper Function: Calculate Cart Features from item names ---
# This function is crucial for creating the 'cart context' for new predictions.
def calculate_cart_features_for_prediction(cart_item_names, item_encoder, scaler):
    """
    Calculates aggregated cart features (avg_price, total_qty, num_unique)
    from a list of item names and scales them using the pre-fitted scaler.

    Args:
        cart_item_names (list): List of item names currently in the cart.
        item_encoder (LabelEncoder): Fitted encoder for item names.
        scaler (MinMaxScaler): Fitted scaler for numerical features.

    Returns:
        dict: A dictionary containing scaled cart features.
    """
    temp_item_data = []
    # Fetch details for each item in the cart from your full item catalog/features
    # In a real system, you'd look these up from an item database.
    # For this example, we'll use a mock item features dictionary.

    for item_name in cart_item_names:
        if item_name in item_features: # Check if item is known
            temp_item_data.append(item_features[item_name])
        else:
            print(f"Warning: Cart item '{item_name}' not found in item features. Skipping.")

    if not temp_item_data:
        # If cart is empty or all items are unknown, use default/average values for features
        # These averages should ideally come from your training data's unscaled means.
        # Here using arbitrary defaults for demonstration.
        avg_price_default = 5.0
        total_qty_default = 1.0
        num_unique_default = 0

        cart_feature_values = {
            'ITEM_PRICE': avg_price_default, # This will be the reference point for the target item's price
            'CART_AVG_ITEM_PRICE': avg_price_default,
            'CART_TOTAL_QUANTITY': total_qty_default,
            'CART_NUM_UNIQUE_ITEMS': num_unique_default
        }
    else:
        cart_df_temp = pd.DataFrame(temp_item_data)
        cart_feature_values = {
            'ITEM_PRICE': cart_df_temp['ITEM_PRICE'].mean(), # Placeholder, this will be replaced by candidate item's price
            'CART_AVG_ITEM_PRICE': cart_df_temp['ITEM_PRICE'].mean(),
            'CART_TOTAL_QUANTITY': cart_df_temp['ITEM_QUANTITY'].sum(),
            'CART_NUM_UNIQUE_ITEMS': cart_df_temp['ITEM_NAME'].nunique() if 'ITEM_NAME' in cart_df_temp.columns else len(cart_item_names) # Use actual name if available
        }

    # Create a dummy DataFrame matching the numerical_features column order for scaling
    numerical_features_order = ['ITEM_PRICE', 'CART_AVG_ITEM_PRICE', 'CART_TOTAL_QUANTITY', 'CART_NUM_UNIQUE_ITEMS']
    dummy_input_for_scaler = pd.DataFrame([cart_feature_values], columns=numerical_features_order)

    scaled_values = scaler.transform(dummy_input_for_scaler).flatten()

    # Return a dictionary of scaled cart features, extracted by name
    return {
        'CART_AVG_ITEM_PRICE': scaled_values[numerical_features_order.index('CART_AVG_ITEM_PRICE')],
        'CART_TOTAL_QUANTITY': scaled_values[numerical_features_order.index('CART_TOTAL_QUANTITY')],
        'CART_NUM_UNIQUE_ITEMS': scaled_values[numerical_features_order.index('CART_NUM_UNIQUE_ITEMS')]
    }

In [ ]:
def dl_recommendations_with_cart(original_customer_id, cart_item_names, num_recommendations=5):
    """
    Generates recommendations for a customer based on their ID and current cart items
    using the trained deep learning model with padded cart item inputs.

    Args:
        original_customer_id (int): The customer's original ID.
        cart_item_names (list): List of item names currently in the user's cart.
        num_recommendations (int): Number of top recommendations to return.

    Returns:
        pd.Series: Recommended item names.
    """
    # 1. Encode Customer ID
    try:
        customer_id_encoded = customer_id_encoder.transform([original_customer_id])[0]
    except ValueError:
        print(f"Customer ID {original_customer_id} not known to the model. Using a default/average customer ID.")
        customer_id_encoded = customer_id_encoder.classes_[0]

    # 2. Identify Candidate Items (not purchased, not in cart)
    purchased_items_encoded = []
    if original_customer_id in train_data_dl['CUSTOMER_ID'].unique():
        purchased_items_encoded = train_data_dl[train_data_dl['CUSTOMER_ID'] == original_customer_id]['ITEM_ID'].tolist()

    cart_item_ids_encoded_unpadded = []
    for item_name in cart_item_names:
        try:
            cart_item_ids_encoded_unpadded.append(item_name_encoder.transform([item_name])[0])
        except ValueError:
            pass # Item not in our vocabulary, ignore

    excluded_items_encoded = set(purchased_items_encoded + cart_item_ids_encoded_unpadded)

    all_known_item_ids_encoded = item_name_encoder.transform(list(item_name_map_dl.values())) # Convert to list
    unseen_item_ids_encoded = [item_id for item_id in all_known_item_ids_encoded if item_id not in excluded_items_encoded]

    if not unseen_item_ids_encoded:
        print(f"User {original_customer_id} has purchased/carted all known items or no unseen items to recommend.")
        return pd.Series([])

    num_unseen = len(unseen_item_ids_encoded)

    # 3. Prepare Context Features for Prediction
    # Pad the cart items for the model input
    padded_cart_items_encoded = pad_cart_items(cart_item_ids_encoded_unpadded, MAX_CART_SIZE)

    # Replicate user-specific and other general context features for each unseen item
    user_context_row = train_data_dl[train_data_dl['CUSTOMER_ID_ENCODED'] == customer_id_encoded].iloc[0] if customer_id_encoded in train_data_dl['CUSTOMER_ID_ENCODED'].unique() else None

    customer_type_for_pred = user_context_row['CUSTOMER_TYPE_ENCODED'] if user_context_row is not None else customer_type_encoder.transform([customer_data['CUSTOMER_TYPE'].mode()[0]])[0]
    channel_for_pred = user_context_row['ORDER_CHANNEL_NAME_ENCODED'] if user_context_row is not None else order_channel_name_encoder.transform([train_data_dl['ORDER_CHANNEL_NAME'].mode()[0]])[0]
    subchannel_for_pred = user_context_row['ORDER_SUBCHANNEL_NAME_ENCODED'] if user_context_row is not None else order_subchannel_name_encoder.transform([train_data_dl['ORDER_SUBCHANNEL_NAME'].mode()[0]])[0]
    occasion_for_pred = user_context_row['ORDER_OCCASION_NAME_ENCODED'] if user_context_row is not None else order_occasion_name_encoder.transform([train_data_dl['ORDER_OCCASION_NAME'].mode()[0]])[0] # Corrected column name

    # Replicate all inputs for prediction
    pred_customer_ids = np.full(num_unseen, customer_id_encoded)
    pred_item_ids = np.array(unseen_item_ids_encoded)

    # Fetch and scale item prices for each candidate item
    candidate_item_prices = []
    for item_id_enc in unseen_item_ids_encoded:
        item_name = item_name_map_dl.get(item_id_enc, f"Unknown_Item_{item_id_enc}")
        price = item_features[item_name]['ITEM_PRICE']
        candidate_item_prices.append(price)

    # Create a dummy DataFrame with only 'ITEM_PRICE' for the scaler, using the same column names as `numerical_features`
    numerical_features_order = ['ITEM_PRICE', 'CART_AVG_ITEM_PRICE', 'CART_TOTAL_QUANTITY', 'CART_NUM_UNIQUE_ITEMS'] # Need all numerical columns for the scaler
    dummy_input_for_scaler = pd.DataFrame(np.zeros((len(candidate_item_prices), len(numerical_features_order))), columns=numerical_features_order)
    dummy_input_for_scaler['ITEM_PRICE'] = candidate_item_prices

    # Also need to calculate and add the cart features for each prediction row
    # These will be the same for all candidate items for a given customer and cart
    cart_features_scaled = calculate_cart_features_for_prediction(cart_item_names, item_name_encoder, scaler)

    dummy_input_for_scaler['CART_AVG_ITEM_PRICE'] = cart_features_scaled['CART_AVG_ITEM_PRICE']
    dummy_input_for_scaler['CART_TOTAL_QUANTITY'] = cart_features_scaled['CART_TOTAL_QUANTITY']
    dummy_input_for_scaler['CART_NUM_UNIQUE_ITEMS'] = cart_features_scaled['CART_NUM_UNIQUE_ITEMS']


    # Scale all numerical features
    scaled_numerical_values = scaler.transform(dummy_input_for_scaler)

    pred_item_prices_scaled = scaled_numerical_values[:, numerical_features_order.index('ITEM_PRICE')]
    pred_cart_avg_item_price_scaled = scaled_numerical_values[:, numerical_features_order.index('CART_AVG_ITEM_PRICE')]
    pred_cart_total_quantity_scaled = scaled_numerical_values[:, numerical_features_order.index('CART_TOTAL_QUANTITY')]
    pred_cart_num_unique_items_scaled = scaled_numerical_values[:, numerical_features_order.index('CART_NUM_UNIQUE_ITEMS')]


    # Replicate general context and cart items
    pred_channels = np.full(num_unseen, channel_for_pred)
    pred_subchannels = np.full(num_unseen, subchannel_for_pred)
    pred_occasions = np.full(num_unseen, occasion_for_pred)
    pred_customer_types = np.full(num_unseen, customer_type_for_pred)

    # Replicate padded cart items for all unseen items
    pred_padded_cart_items = np.tile(np.array(padded_cart_items_encoded), (num_unseen, 1))

    # 4. Create prediction input dictionary for the Keras model
    prediction_inputs = {
        'customer_id_input': pred_customer_ids,
        'item_id_input': pred_item_ids,
        'order_channel_input': pred_channels,
        'order_subchannel_input': pred_subchannels,
        'order_occasion_input': pred_occasions,
        'customer_type_input': pred_customer_types,
        'item_price_input': pred_item_prices_scaled,
        'cart_items_input': pred_padded_cart_items
    }

    # 5. Predict scores
    predicted_quantities = model.predict(prediction_inputs).flatten()

    # Create a Series of scores with item_id_encoded as index
    item_scores = pd.Series(predicted_quantities, index=unseen_item_ids_encoded)

    # 6. Sort and get top N recommendations
    top_n_encoded_item_ids = item_scores.sort_values(ascending=False).head(num_recommendations).index.tolist()

    # Map back to original item names
    recommended_item_names = [item_name_map_dl.get(idx, f"Unknown_Item_{idx}") for idx in top_n_encoded_item_ids]

    return pd.Series(recommended_item_names)

#### Test Questions Evaluation

In [ ]:
test_data.head()

1. Prepare Test Data for Evaluation

In [ ]:
test_cases = []
for index, row in test_data.iterrows():
    customer_id = row['CUSTOMER_ID']
    # Items in the test order (item1, item2, item3) will serve as both
    # the 'cart_items' (context for prediction) and 'actual_purchased_items' (ground truth)
    cart_and_purchased_items = [
        row['item1'], row['item2'], row['item3']
    ]

    # Filter out any NaN values that might exist if an order had less than 3 items
    cart_and_purchased_items = [item for item in cart_and_purchased_items if pd.notna(item)]

    test_cases.append({
        'customer_id': customer_id,
        'cart_items': cart_and_purchased_items,
    })

print(f"Prepared {len(test_cases)} test cases.")

2. Generate Recommendations

In [ ]:
# Generate recommendations for each test case
recommendations = []

for test_case in test_cases:
    customer_id = test_case['customer_id']
    cart_items = test_case['cart_items']

    # Generate top 3 recommendations
    recs = dl_recommendations_with_cart(customer_id, cart_items, num_recommendations=3)

    recommendations.append({
        'customer_id': customer_id,
        'cart_items': cart_items,
        'recommended_items': recs.tolist() # Convert Series to list for easier processing
    })

# Convert recommendations list of dicts to DataFrame
recs_df = pd.DataFrame(recommendations)

# Expand the recommended_items list into separate columns
recs_expanded = pd.DataFrame(recs_df['recommended_items'].tolist(),
                             columns=['RECOMMENDATION 1', 'RECOMMENDATION 2', 'RECOMMENDATION 3'])

# Combine with submission_data
submission_data = pd.concat([test_data.reset_index(drop=True), recs_expanded], axis=1)

print(f"Generated recommendations for {len(recommendations)} test cases.")

In [ ]:
submission_data['RECOMMENDATION 1'].value_counts()

In [ ]:
submission_data['RECOMMENDATION 2'].value_counts()

In [ ]:
submission_data['RECOMMENDATION 3'].value_counts()

In [ ]:
output_data = submission_data[['CUSTOMER_ID', 'ORDER_ID', 'item1', 'item2', 'item3',
             'RECOMMENDATION 1', 'RECOMMENDATION 2', 'RECOMMENDATION 3']].copy()

output_data.insert(output_data.columns.get_loc('item3') + 1, 'item4', 'Missing')

In [ ]:
output_data.to_csv("output_data.csv", index=False)